In [2]:
import quantecon as qe
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import optimize

### Endogenous Grid Method

Problem:

#### Standard VFI Form:

$$V_{n+1}(k_i , z_j) = \max_{k'} \left\{\dfrac{(z_jk_i^\alpha + (1-\delta)k_i - k')^{1-\gamma}}{1-\gamma} + \beta \mathbb{E}( V_n(k',z') | z_j)\right\}$$
$$\text{s.t. } k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

#### EGM Form:

$$V_{n+1}(k , z_j) = \max_{k_i'} \left\{\dfrac{(z_jk^\alpha + (1-\delta)k - k_i')^{1-\gamma}}{1-\gamma} + \beta \mathbb{E}( V_n(k_i',z') | z_j)\right\}$$
$$\text{s.t. } ln(z') = \rho ln(z) + \eta'$$

So in the EGM form, we are creating grid for:
- capital grid for tomorrow's values
- Today's states

To understand why this method is better than standard VFI, we need to look at the FOC that we have to solve in standard VFI and compare it the FOC of EGM:

- Standard VFI: $(z_jk_i^\alpha + (1-\delta)k_i - k')^{-\gamma} = \beta \mathbb{E}(V_k(k',z')|z_j)$

- EGM: $z_k k^\alpha + (1-\delta)k = [\beta \mathbb{E}(V_k(k_i',z')|z_j)]^{\frac{-1}{\gamma}}+ k_i'$ 

Yes, even in EGM we need to solve for k (and it is a non-linear equation) but it is much easier to solve the first one for k' because we don't need to interpolate $\mathbb{E}(V_k(k',z')|z_j)$ for each off grid k' because all the k' s are on the grid (we chose the tomorrow's capital grid). Another beautiful thing is that instead of constantly evaluating expected value for tomorrow as in Standard VFI FOC, in EGM, it is enough to compute it once.

Furthermore, we can even avoid solving this non-linear equation by carefully changing the state variable from the beginning of period to end of period total **cash on hand**

Define: $Y = z_jk^\alpha + (1-\delta)k$

Then Belmann equation becomes:

$$\hat V(Y,z_j) = \max_{k'} \left\{ \dfrac{(Y-k')^{1-\gamma}}{1-\gamma} + \underbrace{\beta \mathbb{E}(\hat V(Y',z')|z_j)}_{\mathbb{V}(k_i',z_j)}\right\}$$
$$\text{s.t. } Y' = z'k'^\alpha + (1-\delta)k'$$
$$ k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

More simply:

$$\hat V(Y,z_j) = \max_{k'} \left\{ \dfrac{(Y-k')^{1-\gamma}}{1-\gamma} + \mathbb{V}(k_i',z_j)\right\}$$
$$\text{s.t. } Y' = z'k'^\alpha + (1-\delta)k'$$
$$ k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

From FOC:

$$ c^*(k_i', z_j)^{-γ}  = \mathbb{V}_k(k_i', z_j)$$

From this equation, obtain $c^*$ and use it in the resource constraint to find $Y^*$:

$$Y^*(k_i',z_j) = c^*(k_i', z_j) + k_i'$$

And with this cash in hands today, we can find LHS of the value function as:

$$\hat V(Y^*(k_i',z_j),z_j) =\dfrac{(c^*(k_i',z_j))^{1-\gamma}}{1-\gamma} + \mathbb{V}(k_i',z_j) $$




In [3]:
# Non stochastic economy:

# Compute the value function for the Non-stochastic economy first, and an obtain the policies. Them we can apply the algorithm for exercise a).

# Define Parameters:

n_k = 500  # Grids for capitals
n_A = 15  # Markov States.
δ = 0.9   # Depreciation.
α = 0.7   # Capital Share.
ρ = 0.98  # Memory of income
σ = 0.00  # Volatility of income.
β = 0.98  # Discont factor.
θ = 1.5   # Expanding grid coefficient.
error = 10e-6 # Error tolerance.
max_iter = 1000

# Construct the markov chain using rouwenhorst method:

markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)

# Probability matrix and states:
# note that element (i,j) of P represents the transition probability
# of going to state "j" given that we are in state "i".

Π = markov.P   
A = markov.state_values  

# Maximum sustainable Capital:

K_max = 2
K_min = 0.1

# Use and expanding grid:

K = K_min + (K_max - K_min) * (np.linspace(0, 1, n_k)**θ)

# Utility function

def U(c):
    if (c>=0).all()>0.0001:
        return -1*((c)**(-1))
    else:
        return np.where(c <= 0.0001, np.nan, -1*(c)**(-1))

V_0 = np.ones((n_A, n_k))
V_1 = np.copy(V_0)
h_1 = np.ones((n_A, n_k), dtype=int)
iter = 0

while iter < max_iter:
    for r in range(n_A):
        for c in range(n_k):
            V_1[r,c] = np.nanmax(U( A[r] * K[c]**α + (1-δ) * K[c] - K) + β * Π[r,:] @ V_0)
            h_1[r,c] = np.nanargmax(U( A[r] * K[c]**α + (1-δ) * K[c] - K) + β * Π[r,:] @ V_0)
    m = 50
    for i in range(m):
        for r in range(n_A):
            for c in range(n_k):
                V_1[r,c] = U( A[r] * K[c]**α + (1-δ) * K[c] - K[h_1[r,c]]) + β * Π[r,:] @ V_1[:,h_1[r,c]]

    if np.amax(np.abs(V_1 - V_0))< error:
        print("Solution Found")
        print(f'Iterations', iter)
        print(f'Error', np.amax(np.abs(V_1 - V_0)))
        break
    else:
        V_0 = np.copy(V_1)
        iter += 1
        if iter >= max_iter:
            print("No solution")

VNSS = V_1.copy()
h_NSS = h_1.copy()

Solution Found
Iterations 20
Error 2.7959231374552473e-07


In [12]:
#Interpolation Economy:

σ = 0.01 # Volatility of income.

# Construct the markov chain using rouwenhorst method:

markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)

# Probability matrix and states:
# note that element (i,j) of P represents the transition probability
# of going to state "j" given that we are in state "i".

Π = markov.P   
A = markov.state_values 

# Initiliaze the new code at the NSS:

V_0 = VNSS.copy()
#V_0 =np.zeros((n_A, n_k))
#h_0 = np.zeros((n_A, n_k))
h_0 = K[h_NSS.copy()]
h_int = np.zeros((n_A, n_k), dtype=float)
V_1 = np.zeros((n_A, n_k))
iter = 0

while iter < max_iter:

    for r in range(n_A):

        E_V = CubicSpline(K, Π[r,:] @ V_0 , bc_type="natural")

        for c in range(n_k):
            
            # First Obtain the interporaled expected value of V, given A today:
            # We have the array for different values for capital, we just need 
            # to get the array of expected values for the value function given
            # the choice for today.          

            def Return_f(kp):
                return -1*(U( A[r] * K[c]**α + (1-δ) * K[c] - kp) + β * E_V(kp))

            def FOC(kp):
                Δ = 10e-5
                return ((-Return_f(kp + Δ))-(-Return_f(kp - Δ)))/(2*Δ)

            #if FOC(K_max)>0:
            #    print("Grid for capital is too small")

            # We don't want to allow for states where consumption is negative:

            def ieq_neg(kp):
                return (A[r] * K[c]**α + (1-δ) * K[c] - kp)
            constraints_1 = {'type': 'ineq', 'fun': ieq_neg}
            
            # Setting the minimum capital in the continuous grid. 
            # note that this is les than K_min.

            def ieq_min(kp):
                return   kp - (K_min * 0.9)
            constraints_2 = {'type': 'ineq', 'fun': ieq_min}
           
            # Setting the maximum capital in the continuous grid. 
            # note that this is greater than K_max. 

            def ieq_max(kp):
                return - kp + (1.1 * K_max)
            constraints_3 = {'type': 'ineq', 'fun': ieq_max}
            
            h_int[r,c] = scipy.optimize.minimize(Return_f, K_min, constraints=[constraints_1, constraints_2, constraints_3], tol = 10e-7).x[0]
            
            V_1[r,c] = U( A[r] * K[c]**α + (1-δ) * K[c] - h_int[r,c]) + β * E_V(h_int[r,c])
    
    #Howard steps:
    
    m = 30
    for i in range(m):
        for r in range(n_A):
            E_V = CubicSpline(K, Π[r,:] @ V_1 , bc_type="natural")
            for c in range(n_k):
                V_1[r,c] = U( A[r] * K[c]**α + (1-δ) * K[c] - h_int[r,c]) + β * E_V(h_int[r,c])

    
    if np.amax((h_int - h_0)/(1 + np.abs(h_0)))< error:
        print("Solution Found")
        break

    else:
        print(iter)
        #print(np.amax((h_int - h_0)/(1 + np.abs(h_0))))
        iter += 1
        V_0 = V_1.copy()
        h_0 = h_int.copy()
        

In [15]:
# Define Parameters:

n_k = 500  # Grids for capitals
n_A = 15  # Markov States.
δ = 0.9   # Depreciation.
α = 0.7   # Capital Share.
ρ = 0.98  # Memory of income
σ = 0.01  # Volatility of income.
β = 0.98  # Discont factor.
θ = 1.5   # Expanding grid coefficient.
error = 10e-6 # Error tolerance.
max_iter = 1000

markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)

Π = markov.P   
A = markov.state_values 

# Maximum sustainable Capital:

K_max = 2
K_min = 0.1

# Use and expanding grid:

Kp = K_min + (K_max - K_min) * (np.linspace(0, 1, n_k)**θ)


#Initial guess
V_0 = np.zeros((n_A,n_k))
for i in range(n_A):
    for j in range(n_k):
        V_0[i,j] = Kp[j]**(α) * A[i]

#Some necessary matrices for computation
V_k = np.zeros((n_A,n_k))
c_star = np.zeros((n_A,n_k))
Y_star = np.zeros((n_A,n_k))
value = np.zeros((n_A,n_k))

Y_prime = np.zeros((n_A,n_k))


dist = 1000
iterasyon = 0

delta = 10e-4

while dist > error and iterasyon < max_iter:
    for j in range(n_A):
        V_spline = CubicSpline(Kp,V_0[j,:])
        c_star[j,:] = V_spline(Kp + delta) - V_spline(Kp - delta)/ (2*delta)

    Y_star = c_star**(-1/2) + Kp

    Values = -c_star **(-1) + V_0

    for j in range(n_A):
        Y_star[]

# while dist > error:
#     for i in range(n_k):
#         for j in range(n_A):
#             #Step 1: Find c_star[i,j]
#             V_spline = CubicSpline(Kp,V_0[:,j])
#             V_k[i,j] = (V_spline(Kp[i]+10e-2) - V_0[i,j])/10e-4
#             c_star[i,j] = V_k[i,j]**(-1/2)     
            
#             #Step 2: Obtain today's end of period resources
#             Y_star[i,j] = c_star[i,j] + Kp[i]
#             value[j] = -c_star[i,j]**(-1) + V_0[i,j]

#         for j in range(n_A):
#             #Step 3: Interpolating for Y'
#             Y_star_sorted_index = np.argsort(Y_star[i,:])
#             value_sorted = value[Y_star_sorted_index]
#             sorted_y = np.sort(Y_star[i,:])

#             V_tilde = CubicSpline(sorted_y,value_sorted)   
#             V_1[i,j] = β * np.dot(Π[j,:], V_tilde(A*(Kp[i]**(α)) +(1-δ)*Kp[i]))                       
    
#     dist = np.max(np.abs(V_1 - V_0))
#     iterasyon = iterasyon + 1
#     print('iteration: ', iterasyon)
#     print('dist: ', dist)
#     V_0 = V_1.copy()

IndexError: index 15 is out of bounds for axis 0 with size 15

In [13]:
a = np.array([3,5,9,8])
b = np.array([1,2,3,4])

In [14]:
b[np.argsort(a)]

array([1, 2, 4, 3])